In [1]:
import gensim
import keras
import numpy as np
from CMVscraper_script_delta import *
from detect_and_score import *

c:\users\paul\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


Load Google's word2vec model

In [2]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format ('models/GoogleNews-vectors-negative300.bin', binary=True)  

In [3]:
#word2vec.similarity("cat", "dog")

Load in our own Keras LSTM models

In [4]:
detect_model = keras.models.load_model("models/bilstm_cos.h5")

In [5]:
#model2 = keras.models.load_model("models/bilstm_evidence.h5")

In [6]:
score_model = keras.models.load_model("models/argument_quality.h5")

In [9]:
OPname, OPtxt, CommentText, commentName, URL, TOPIC, deltalink, deltabot = search("Twitch")
#OPname, OPtxt, CommentText, commentName, URL, TOPIC, deltalink, deltabot = urlSearch(20)
deltaNames, deltaComments = getDeltaNames(deltalink, deltabot, CommentText)
delta_post = find_delta_txt(deltaNames, deltaComments, CommentText, commentName)

TypeError: 'NoneType' object is not iterable

In [ ]:
topic = TOPIC.split(":")[1]
topic

In [ ]:
OP_post = OPtxt
OP_post

In [ ]:
OP_arguments = detect_and_score(detect_model, score_model, OP_post, topic, word2vec)
comment_arguments = detect_and_score(detect_model, score_model, CommentText[1], topic, word2vec)
delta_arguments = detect_and_score(detect_model, score_model, delta_post[0], topic, word2vec)

In [ ]:
calculate_score(OP_arguments)

In [ ]:
calculate_score(comment_arguments)

In [ ]:
calculate_score(delta_arguments)

In [ ]:
delta_post[0]